# Week 3 Homework: Working with Pandas

Create a new IPython Notebook (or copy this one) and use it to answer the following assignments, using what you've learned this week. Note that the last sections for each are meant to be a little more challenging.

## Question 1

Load the MovieData.csv dataset into a pandas DataFrame as described in this week's lesson, and use it to find the following values:

a. What is the median profit of movies with budgets of over $50M?

b. How many movies were released by each film distributor? Output the results to a csv file.

c. What are the mean and median movie profits by decade? Which decade was the most profitable? (Hint: Answering this question requires several steps: grouping the movies by decade, computing the mean and median profits for each decade, and combining the results back together.)

## Question 2

Load the earthquake data in QuakeData.csv into a DataFrame, and use it to answer the following questions:

a. What is the median earthquake magnitude?

b. What is the correlation between magnitude and depth?

c. What fraction (not count) of earthquakes happen each month, across all years (i.e. all earthquakes occurring in January as a proportion of the grand total, all earthquakes in February as a proportion of the grand total, etc.)?

d. Is there correlation between the number of movies released monthly (i.e. Jan-1990, Feb-1990... ) , and the number of earthquakes in that month?

## Question 1
### a)

In [1]:
import pandas as pd 
import datetime as dt

In [2]:
movies = pd.read_csv('moviedata.csv', sep='\t')

In [3]:
movies.head()

,Release_Date,Movie,Distributor,Budget,US Gross,Worldwide Gross
0,03/09/12,John Carter,NaN,300000000,66439100,254439100
1,05/25/07,Pirates of the Caribbean: At World's End,Buena Vista,300000000,309420425,960996492
2,12/13/13,The Hobbit: There and Back Again,New Line,270000000,Unknown,Unknown
3,12/14/12,The Hobbit: An Unexpected Journey,New Line,270000000,Unknown,Unknown
4,11/24/10,Tangled,Buena Vista,260000000,200821936,586581936


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3627 entries, 0 to 3626
Data columns (total 6 columns):
Release_Date       3627 non-null object
Movie              3627 non-null object
Distributor        2968 non-null object
Budget             3627 non-null int64
US Gross           3627 non-null object
Worldwide Gross    3627 non-null object
dtypes: int64(1), object(5)
memory usage: 170.1+ KB


In [5]:
movies.dtypes

Release_Date       object
Movie              object
Distributor        object
Budget              int64
US Gross           object
Worldwide Gross    object
dtype: object

In [6]:
# US Gross and Worldwide Gross is supposed to be int. But showing as string.
movies['US Gross'].unique()

array(['66439100', '309420425', 'Unknown', ..., '48482', '1338', '181041'],
      dtype=object)

In [7]:
movies['Worldwide Gross'].unique()

array(['254439100', '960996492', 'Unknown', ..., '71644', '240495',
       'Unkno'], dtype=object)

In [8]:
movies = pd.read_csv('moviedata.csv', sep = '\t', na_values=['Unknown', 'Unkno'])

In [9]:
movies.dtypes

Release_Date        object
Movie               object
Distributor         object
Budget               int64
US Gross           float64
Worldwide Gross    float64
dtype: object

In [10]:
# Cool, now it's showing correctly. 

In [11]:
# Now we need to parse the dates using pandas parse_dates parameter.
movies = pd.read_csv('moviedata.csv', sep = '\t', na_values=['Unknown', 'Unkno'], 
                    parse_dates=[0])

In [12]:
print(movies['Release_Date'].max())
print(movies['Release_Date'].min())

2068-12-11 00:00:00
1969-01-01 00:00:00


In [13]:
# Need to convert date time datetime object.
# Something off about the date.
# I'll create a function to convert to correct century. 

def make_date(date_str):
    """
    Turn a mm/dd/yy string into a datetime object"""
    m, d, y = date_str.split("/")
    m = int(m)
    d = int(d)
    y = int(y)
    if y > 13:
        y += 1900
    else: 
        y += 2000
    return dt.datetime(y, m, d)

In [14]:
movies = pd.read_csv('moviedata.csv', sep='\t', na_values = ['Unkno', 'Unknown'], 
                    parse_dates=[0], date_parser=make_date)

In [15]:
print(movies['Release_Date'].max())
print(movies['Release_Date'].min())

2013-12-13 00:00:00
1915-02-08 00:00:00


In [16]:
movies.head()

,Release_Date,Movie,Distributor,Budget,US Gross,Worldwide Gross
0,2012-03-09,John Carter,NaN,300000000,66439100.0,254439100.0
1,2007-05-25,Pirates of the Caribbean: At World's End,Buena Vista,300000000,309420425.0,960996492.0
2,2013-12-13,The Hobbit: There and Back Again,New Line,270000000,NaN,NaN
3,2012-12-14,The Hobbit: An Unexpected Journey,New Line,270000000,NaN,NaN
4,2010-11-24,Tangled,Buena Vista,260000000,200821936.0,586581936.0


In [17]:
movies["Worldwide Gross"].isnull()

0       False
1       False
2        True
3        True
4       False
5       False
6       False
7       False
8        True
9       False
10      False
11      False
12      False
13      False
14       True
15       True
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3597     True
3598     True
3599     True
3600     True
3601    False
3602     True
3603     True
3604     True
3605     True
3606     True
3607     True
3608    False
3609     True
3610     True
3611     True
3612     True
3613     True
3614     True
3615     True
3616     True
3617     True
3618     True
3619     True
3620    False
3621    False
3622    False
3623     True
3624    False
3625     True
3626     True
Name: Worldwide Gross, Length: 3627, dtype: bool

I see NaN values in the Worldwide Gross column and I need them to contain
value. I will forward the value of the US Gross into the Worldwide Gross.
Some movies may have not received Worldwide showings. First, I need to 
fill na values with 0. So that I can do subtraction to get the column I need.


In [18]:
movies.fillna(0, inplace=True)

In [19]:
movies.head()

,Release_Date,Movie,Distributor,Budget,US Gross,Worldwide Gross
0,2012-03-09,John Carter,0,300000000,66439100.0,254439100.0
1,2007-05-25,Pirates of the Caribbean: At World's End,Buena Vista,300000000,309420425.0,960996492.0
2,2013-12-13,The Hobbit: There and Back Again,New Line,270000000,0.0,0.0
3,2012-12-14,The Hobbit: An Unexpected Journey,New Line,270000000,0.0,0.0
4,2010-11-24,Tangled,Buena Vista,260000000,200821936.0,586581936.0


In [20]:
# Forwards values from US Gross to Worldwide Gross where value is 0
movies["Worldwide Gross"][movies["Worldwide Gross"]==0] = movies["US Gross"]

C:\Users\Khemra\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


I want to add a profit column since that's what the question is asking. 


In [21]:
movies["Profit"] = movies["Worldwide Gross"] -  movies["Budget"]

In [22]:
movies.head()

,Release_Date,Movie,Distributor,Budget,US Gross,Worldwide Gross,Profit
0,2012-03-09,John Carter,0,300000000,66439100.0,254439100.0,-45560900.0
1,2007-05-25,Pirates of the Caribbean: At World's End,Buena Vista,300000000,309420425.0,960996492.0,660996492.0
2,2013-12-13,The Hobbit: There and Back Again,New Line,270000000,0.0,0.0,-270000000.0
3,2012-12-14,The Hobbit: An Unexpected Journey,New Line,270000000,0.0,0.0,-270000000.0
4,2010-11-24,Tangled,Buena Vista,260000000,200821936.0,586581936.0,326581936.0


movies.tail()

In [23]:
# Data and values look correct. 

Now for the answer to the question. Anything greater than $50,000,000

In [24]:
big_budget = movies[movies.Budget > 50000000]

In [25]:
big_budget.Profit.median()

89246220.0

## b. How many movies were released by each film distributor? Output the results to a csv file.

To answer this question we use the groupby function. From what I understand, I think grouby in Pandas is simliar to a Dictionary in Python. 

In [27]:
distributors = movies.groupby("Distributor").aggregate(len)
distributor_count = distributors['Movie']

In [29]:
# Using aggregate(len) is telling what function to use on each group. 
# instead of using the len() function. 

# Here is the Series. 
distributor_count

Distributor
0                                  659
20th Century Fox                   230
3D Entertainment                     1
8 X Entertainment                    1
ART                                  1
Access                               1
Alliance                             4
American International Pictures      1
Anchor Bay                           4
Apparition                           4
Artisan                             23
Artistic License                     1
Atlantic                             1
Attitude Films                       1
Avatar                               1
Avco Embassy                         5
Barking Cow                          1
Big Pictures                         1
Bigger Picture                       1
Black Diamond Pictures               1
Buena Vista                        227
CBS Films                            3
CFP                                  1
CHRIST                               1
Cannon                               4
Cinema Servic

Output to CSV file. 

In [30]:
# Pandas has a to_csv() method. 
distributor_count.to_csv("distributor__counts.csv")

C:\Users\Khemra\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


## c. What are the mean and median movie profits by decade? Which decade was the most profitable? (Hint: Answering this question requires several steps: grouping the movies by decade, computing the mean and median profits for each decade, and combining the results back together.)

Need to get the decade of each movie. I'll use division and subraction to get the decade. For example, 1969 % 10 = 9 then 1969 - 9 = 1960

In [31]:
# Create a function to get the decade. 
def get_decade(date):
    year = date.year
    y = year % 10 # remainder of dividing by 10
    return year - y

Apply the function to the date column and put results into a new column. 

In [32]:
movies["Decade"] = movies.Release_Date.apply(get_decade)

Check to see if it's correct. 

In [33]:
movies.tail()

,Release_Date,Movie,Distributor,Budget,US Gross,Worldwide Gross,Profit,Decade
3622,2006-05-26,Cavite,Truly Indie,7000,70071.0,71644.0,64644.0,2000
3623,2004-01-01,The Mongol King,CustomFlix,7000,900.0,900.0,-6100.0,2000
3624,1999-04-04,Following,Zeitgeist,6000,48482.0,240495.0,234495.0,1990
3625,2005-07-13,Return to the Land of Wonders,0,5000,1338.0,1338.0,-3662.0,2000
3626,2005-08-05,My Date With Drew,0,1100,181041.0,181041.0,179941.0,2000


In [34]:
# Now we create a grouping of the movies by the decades. 
by_decade = movies.groupby("Decade")

In [35]:
decade_mean_profit = by_decade['Profit'].mean()
decade_med_profit = by_decade['Profit'].median()

In [37]:
# Now we put these Series together into a new DF. 
decades = pd.DataFrame({"Mean Profit": decade_mean_profit, "Median Profit": decade_med_profit})
decades

,Mean Profit,Median Profit
Decade,,
1910,6.101364e+06,7800000.0
1920,6.346800e+06,3979000.0
1930,3.892876e+07,2265500.0
1940,1.025301e+07,6012000.0
1950,1.816625e+07,8690000.0
1960,2.845890e+07,10564923.0
1970,6.358547e+07,19533200.0
1980,5.114162e+07,16168359.0
1990,5.751548e+07,9133087.0


# Question 2 

Load the earthquake data in QuakeData.csv into a DataFrame, and use it to answer the following questions:

a. What is the median earthquake magnitude?

In [39]:
quakes = pd.read_csv("QuakeData.csv", parse_dates=[0])

In [40]:
quakes.dtypes

DateTime      datetime64[ns, UTC]
Latitude                  float64
Longitude                 float64
Depth                     float64
Magnitude                 float64
MagType                    object
NbStations                  int64
Gap                         int64
Distance                  float64
RMS                       float64
Source                     object
EventID                    object
Version                     int64
dtype: object

In [42]:
# Answer
quakes.Magnitude.median()

4.5

b. What is the correlation between magnitude and depth?

In [43]:
# Use the .corr() method on the entire DF to give correlations between all columns. 

In [44]:
quakes.corr()

,Latitude,Longitude,Depth,Magnitude,NbStations,Gap,Distance,RMS,Version
Latitude,1.000000,0.151126,-0.152737,-0.094021,0.169994,-0.002338,NaN,-0.059844,0.028847
Longitude,0.151126,1.000000,-0.056669,0.094502,-0.062045,-0.158388,NaN,-0.162260,-0.031297
Depth,-0.152737,-0.056669,1.000000,0.029175,0.130701,-0.132728,NaN,-0.100127,-0.014670
Magnitude,-0.094021,0.094502,0.029175,1.000000,0.703717,-0.523315,NaN,-0.012080,0.106513
NbStations,0.169994,-0.062045,0.130701,0.703717,1.000000,-0.483381,NaN,-0.080913,0.114673
Gap,-0.002338,-0.158388,-0.132728,-0.523315,-0.483381,1.000000,NaN,-0.010979,-0.030786
Distance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RMS,-0.059844,-0.162260,-0.100127,-0.012080,-0.080913,-0.010979,NaN,1.000000,0.018472
Version,0.028847,-0.031297,-0.014670,0.106513,0.114673,-0.030786,NaN,0.018472,1.000000


In [45]:
# Or use corr() between two columns only. 
quakes.Depth.corr(quakes.Magnitude)

0.02917515915997664

c. What fraction (not count) of earthquakes happen each month, across all years (i.e. all earthquakes occurring in January as a proportion of the grand total, all earthquakes in February as a proportion of the grand total, etc.)?

Need to count the quakes that happened each month then divide each count by the total number of quakes. First, need to find the month for each quake. Only need the month from the dates.

In [46]:
quakes['Month'] = quakes.DateTime.apply(lambda x: x.month)

In [47]:
quakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID,Version,Month
0,2012-01-01 00:30:08.770000+00:00,12.008,143.487,35.0,5.1,mb,178,45,NaN,1.20,pde,pde20120101003008770_35,1363392487731,1
1,2012-01-01 00:43:42.770000+00:00,12.014,143.536,35.0,4.4,mb,29,121,NaN,0.98,pde,pde20120101004342770_35,1363392488431,1
2,2012-01-01 00:50:08.040000+00:00,-11.366,166.218,67.5,5.3,mb,143,43,NaN,0.82,pde,pde20120101005008040_67,1363392488479,1
3,2012-01-01 01:22:07.660000+00:00,-6.747,130.008,145.0,4.2,mb,14,112,NaN,1.16,pde,pde20120101012207660_145,1363392488594,1
4,2012-01-01 02:35:21.110000+00:00,23.472,91.834,27.8,4.6,mb,74,77,NaN,0.65,pde,pde20120101023521110_27,1363392488611,1


Use the groupby() to group by month. Count entries with len and get some column counts

In [48]:
monthly_counts = quakes.groupby("Month").aggregate(len)["Latitude"]

In [49]:
monthly_counts

Month
1     1024.0
2     1081.0
3     1145.0
4     1393.0
5     1058.0
6      900.0
7      882.0
8     1022.0
9     1132.0
10    1051.0
11     996.0
12    1000.0
Name: Latitude, dtype: float64

In [50]:
# Now divide the counts series by the series sum. 

monthly_fractions = monthly_counts / (1.0 * monthly_counts.sum())

In [51]:
monthly_fractions

Month
1     0.080732
2     0.085225
3     0.090271
4     0.109823
5     0.083412
6     0.070956
7     0.069536
8     0.080574
9     0.089246
10    0.082860
11    0.078524
12    0.078839
Name: Latitude, dtype: float64

d. Is there correlation between the number of movies released monthly (i.e. Jan-1990, Feb-1990... ) , and the number of earthquakes in that month?

Need to count both movies and quakes across all calendar months, or month-year combinations. Then we need to correlate the resulting series to each other. 

In [52]:
# A function that accepts a date and returns the first day of the month. 
# This way all date-times in that month map to a common date. 

def make_month(date):
    return dt.datetime(year=date.year, month=date.month, day=1)

In [53]:
# Now use the function on both Movies and Quake datasets and get monthly count Series.

In [54]:
movies['Month'] = movies.Release_Date.apply(make_month)

In [55]:
quakes['Month'] = quakes.DateTime.apply(make_month)

In [56]:
movies_per_month = movies.groupby("Month")["Movie"].aggregate(len)

In [57]:
quakes_per_month = quakes.groupby("Month")["MagType"].aggregate(len)

In [58]:
quakes_per_month

Month
2012-01-01    1005
2012-02-01    1081
2012-03-01    1145
2012-04-01    1393
2012-05-01    1058
2012-06-01     900
2012-07-01     882
2012-08-01    1022
2012-09-01    1132
2012-10-01    1051
2012-11-01     996
2012-12-01    1000
2013-01-01      19
Name: MagType, dtype: int64

In [59]:
movies_per_month

Month
1915-02-01     1
1916-09-01     1
1916-12-01     1
1920-09-01     1
1925-01-01     1
1925-12-01     1
1927-08-01     1
1929-06-01     1
1930-01-01     1
1931-12-01     1
1933-02-01     1
1933-03-01     1
1933-04-01     1
1934-01-01     1
1935-09-01     1
1936-02-01     1
1936-06-01     1
1936-10-01     1
1937-12-01     1
1938-01-01     2
1939-01-01     2
1939-12-01     1
1940-01-01     2
1940-11-01     1
1941-10-01     1
1942-11-01     1
1943-01-01     1
1943-12-01     1
1944-01-01     1
1944-08-01     1
              ..
2010-05-01    13
2010-06-01    18
2010-07-01    14
2010-08-01    12
2010-09-01    15
2010-10-01    24
2010-11-01    16
2010-12-01    25
2011-01-01     7
2011-02-01     4
2011-03-01    11
2011-04-01    15
2011-05-01    16
2011-06-01    10
2011-07-01    14
2011-08-01    21
2011-09-01    20
2011-10-01    14
2011-11-01    13
2011-12-01     9
2012-01-01    11
2012-02-01     5
2012-03-01     7
2012-04-01     1
2012-05-01     1
2012-06-01     2
2012-07-01     1
2012-11-

In [60]:
quakes_per_month.corr(movies_per_month)

0.016073604515738773